In [ ]:
%pylab inline

In [ ]:
import os
import sys
import time 

import numpy as np
import pylab as plt 
import h5py

sys.path.insert(0,'..')

from motor.motor import MotorsPool
from shutter.shutter import Shutter
from detector.xiApi import Detector

In [ ]:
shutter = Shutter('COM7', 4)
print shutter.is_open()
shutter.open()
print shutter.is_open()
time.sleep(0.5)
shutter.close()
print shutter.is_open()

In [ ]:
detector = Detector()
detector.enable_cooling()

def show_frame(exposure):
    shutter.open()
    detector.set_exposure(exposure)
    res = detector.get_image()
    shutter.close()
    plt.figure()
    plt.imshow(res, cmap=plt.cm.gray)
    plt.colorbar(orientation='horizontal');

show_frame(1*1000)

In [ ]:
motors_pool = MotorsPool()

In [ ]:
for motor in motors_pool.motors:
    print motor.name, motor.device_id
    print motor.get_status()
    print motor.get_speed_acceleration()
    motor.set_speed_acceleration(500, 500)
    print motor.get_speed_acceleration()

motor_shift = motors_pool.motors[0]
motor_rotate = motors_pool.motors[1]

In [ ]:
one_deg_steps = 32300./360
empty_shift = -4200

In [ ]:
motor_shift.move_to_position(0)
print motor_shift.get_status()
print motor_shift.get_speed_acceleration()

In [ ]:
show_frame(1*1000)

In [ ]:
motor_rotate.move_to_position(0)
print motor_rotate.get_status()

In [ ]:
def get_data_frame(exposure):
    shutter.open()
    detector.set_exposure(exposure)
    res = detector.get_image()
    shutter.close()
    return res

def get_dark_frame(exposure):
    detector.set_exposure(exposure)
    res = detector.get_image()
    shutter.close()
    return res

In [ ]:
exposure = 10e3
empty_frames_count = 10
dark_frames_count = 10
print exposure
data_angles = np.arange(0,181,1)
print data_angles

In [ ]:
motor_rotate.move_to_position(0)
print motor_rotate.get_status()

motor_shift.move_to_position(0)
print motor_shift.get_status()

print 'Starting collecting data' 
for iangle, angle in enumerate(data_angles):
    print angle, 
    motor_rotate.move_to_position(angle*one_deg_steps)
    frame = get_data_frame(exposure)
    with h5py.File('test_tomo.h5') as h5f:
        frame_name = '/data/{:04d}/data'.format(iangle)
        h5f.create_dataset(frame_name, data=frame, compression="gzip", chunks=True)
        h5f['/data/{:04d}/motor_rotation_position'.format(iangle)] = motor_rotate.get_position()
        h5f['/data/{:04d}/motor_shift_position'.format(iangle)] = motor_shift.get_position()
    print 
    
print 'Starting collecting empty'
motor_shift.move_to_position(empty_shift)
for iangle in  range(empty_frames_count):
    print iangle,
    frame = get_data_frame(exposure)
    with h5py.File('test_tomo.h5') as h5f:
        frame_name = '/empty/{:04d}/frame'.format(iangle)
        h5f.create_dataset(frame_name, data=frame, compression="gzip", chunks=True)
        h5f['/empty/{:04d}/motor_rotation_position'.format(iangle)] = motor_rotate.get_position()
        h5f['/empty/{:04d}/motor_shift_position'.format(iangle)] = motor_shift.get_position()
    print 
    
print 'Starting collecting dark'
for iangle in  range(dark_frames_count):
    print iangle,
    frame = get_dark_frame(exposure)
    with h5py.File('test_tomo.h5') as h5f:
        frame_name = '/dark/{:04d}/frame'.format(iangle)
        h5f.create_dataset(frame_name, data=frame, compression="gzip", chunks=True)
        h5f['/dark/{:04d}/motor_rotation_position'.format(iangle)] = motor_rotate.get_position()
        h5f['/dark/{:04d}/motor_shift_position'.format(iangle)] = motor_shift.get_position()
    print

motor_rotate.move_to_position(0)
print motor_rotate.get_status()

motor_shift.move_to_position(0)
print motor_shift.get_status()

In [ ]:
print motor_rotate.get_status()
print motor_shift.get_status()

In [ ]:
#!rm test_tomo.h5

In [ ]:
motor_shift.move_to_position(0)

In [ ]:
empty_shift